# Start

In [2]:
import sys
from importlib import reload

import pandas as pd

import fraud_detection as fd

# Add the src directory to the system path to import modules
reload(fd)
sys.path.append('../src')

In [3]:
datapath = "../data/transformed_label_and_damage.parquet"

# Model Vergleich

Vergleich verschiedener Modelle mit Cross Validition auf identischen Splits.

Die Combinded Modelle bestehen aus einem Klassifikator und einem Regressor, die zusammenarbeiten, um eine bessere Vorhersage zu erzielen.

In [ ]:
%%capture

X, _ = fd.data_loader.load_data_np(datapath)
neural_net = fd.neuralnets.train_nn.getNN(X.shape[1])

models = [
    ("LightGBM", fd.models.get_lgmb()),
    ("LightGBM Combined", fd.models.models.get_lgmb_clf_with_reg()),
    ("XGBoost", fd.models.get_xgb(10)),
    ("XGBoost Combined", fd.models.models.get_xgb_clf_with_reg(10)),
    ("Neural Net", neural_net),
]

model_metrics = fd.model_comparison.compare_models(models, datapath, n_splits=5)

In [7]:
model_metrics_df = pd.DataFrame(model_metrics)
model_metrics_df.T

,precision,recall,f1,damage_total,damage_prevented,damage_missed,detected bonus,fp penalty,Bewertung,cm
LightGBM,0.798154,0.556331,0.655593,6015.164,3594.838,2420.326,2380.0,1204.0,-1244.326,"[[28553.4, 120.4], [379.6, 476.0]]"
LightGBM Combined,0.816027,0.545581,0.653178,6015.164,3587.24,2427.924,2334.0,1066.0,-1159.924,"[[28567.2, 106.6], [388.8, 466.8]]"
XGBoost,0.865924,0.513788,0.644868,6015.164,3350.516,2664.648,2198.0,680.0,-1146.648,"[[28605.8, 68.0], [416.0, 439.6]]"
XGBoost Combined,0.840405,0.538572,0.656195,6015.164,3528.006,2487.158,2304.0,880.0,-1063.158,"[[28585.8, 88.0], [394.8, 460.8]]"
Neural Net,0.887073,0.509126,0.645705,6015.164,3340.452,2674.712,2178.0,552.0,-1048.712,"[[28618.6, 55.2], [420.0, 435.6]]"


Das neuronale Netzwerk ist nocht recht unstabil und erzielt nicht immer gute Ergebnisse (in diesem Durchlauf schnitt es aber am besten ab).